In [2]:
import numpy as np
import pandas as pd
import pickle
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error


In [3]:
def proc_col(col, train_col=None):
    """
    Assigns unique continuous IDs to the values in a pandas column.
    """
    if train_col is not None:
        uniq = train_col.unique()
    else:
        uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx.get(x, -1) for x in col]), len(uniq)

def encode_data(df, train=None):
    """
    Transforms rating data by assigning continuous IDs to users and movies. 
    If a training dataset is provided, it applies the same encoding scheme as used in the training data.
    """
    df = df.copy()
    for col_name in ["userId", "resId"]:
        train_col = None
        if train is not None:
            train_col = train[col_name]
        _,col,_ = proc_col(df[col_name], train_col)
        df[col_name] = col
        df = df[df[col_name] >= 0]
    return df

In [4]:
# path = "../senti_prediction_result/"
# data = []
# for file in os.listdir(path):
#     with open(path + file, "rb")as f:
#         temp = pickle.load(f)
#         data += [pair for items in temp for pair in items]

# df_data = []
# errors = []
# for id, item in enumerate(data):
#     try:
#         df_data.append((item[0], item[1], str(item[2])))
#     except:
#         errors += [(id, item)]

# df_sent = pd.DataFrame(df_data, columns=["user_id", "business_id", "sentiment"])
# original_df = pd.read_csv("reviews_restaurant_indexed.csv")
# df = pd.merge(original_df, df_sent, on=["user_id", "business_id"], how="inner")
# df = encode_data(df)

# df.to_csv("reviews_restaurant_indexed_sent.csv", index=False)

In [4]:
# Generate this file using the previous cell if needed
df = pd.read_csv("reviews_restaurant_indexed_sent.csv")
df = encode_data(df)
df.head()

review_id                 user_id  userId  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA       0   
1  VJxlBnJmCDIy8DFG0kjSow  Iaee7y6zdSB3B-kRCo4z1w       1   
2  S6pQZQocMB1WHMjTRbt77A  ejFxLGqQcWNLdNByJlIhnQ       2   
3  WqgTKVqWVHDHjnjEsBvUgg  f7xa0p_1V9lx53iIGN5Sug       3   
4  M0wzFFb7pefOPcxeRVbLag  dCooFVCk8M1nVaQqcfTL3Q       4   

              business_id  resId  rating  sentiment  mf_rating  
0  XQfwVwDr-v0ZS3_CbbE5Xw      0     3.0      3.998   3.866407  
1  XQfwVwDr-v0ZS3_CbbE5Xw      0     2.0      3.189   2.745549  
2  XQfwVwDr-v0ZS3_CbbE5Xw      0     4.0      4.841   3.159871  
3  XQfwVwDr-v0ZS3_CbbE5Xw      0     3.0      1.915   3.195579  
4  XQfwVwDr-v0ZS3_CbbE5Xw      0     2.0      3.641   3.148946

In [9]:
num_users = len(df.userId.unique())
num_items = len(df.resId.unique())

# model_mf = MF(num_users, num_items)
# model_cn = CollabFNet(num_users, num_items)

# # Load the state dictionary
# model_mf.load_state_dict(torch.load('model/mf-model-no-bias.pt'))
# model_cn.load_state_dict(torch.load('model/collabfnet-model.pt'))

In [6]:
users = torch.LongTensor(df.userId.values)
items = torch.LongTensor(df.resId.values)
ratings = torch.FloatTensor(df.rating.values)
ratings = ratings.unsqueeze(1)

In [ ]:
# model_mf = torch.load("model/mf-model-no-bias.pt", weights_only=False)
# model_mf.eval()
# model_mf = model_mf.cpu()

# y_hat_mf = model_mf(users, items)
# y_hat_mf = y_hat_mf.detach().numpy()

# y_hat_mf[y_hat_mf > 5] = 5
# y_hat_mf[y_hat_mf < 1] = 1

# df["mf_rating"] = y_hat_mf


In [ ]:
# # Note here there is no matrix multiplication, we could potentially make the embeddings of different sizes.
# # Here we could get better results by keep playing with regularization.
    
# class CollabFNet(nn.Module):
#     def __init__(self, num_users, num_items, emb_size=100, n_hidden=10):
#         super(CollabFNet, self).__init__()
#         self.user_emb = nn.Embedding(num_users, emb_size)
#         self.item_emb = nn.Embedding(num_items, emb_size)
#         self.lin1 = nn.Linear(emb_size*2, n_hidden)
#         self.lin2 = nn.Linear(n_hidden, 1)
#         self.drop1 = nn.Dropout(0.1)
        
#     def forward(self, u, v):
#         U = self.user_emb(u)
#         V = self.item_emb(v)
#         x = F.relu(torch.cat([U, V], dim=1))
#         x = self.drop1(x)
#         x = F.relu(self.lin1(x))
#         x = self.lin2(x)
#         return x
    
# model_cn = CollabFNet(num_users, num_items, emb_size=100)

# model_cn = torch.load("model/collabfnet-model.pt", weights_only=False)


In [ ]:
# model_cn.eval()
# model_cn = model_cn.cpu()
# y_hat_cn = model_cn(users, items)
# y_hat_cn = y_hat_cn.detach().numpy()

# y_hat_cn[y_hat_cn > 5] = 5
# y_hat_cn[y_hat_cn < 1] = 1

# df["colab_rating"] = y_hat_cn

# df.to_csv("reviews_restaurant_indexed_sent.csv", index=False)

In [14]:
betas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
mse = []
for beta in betas:
    total_pred = df["sentiment"] * beta + df["mf_rating"] * (1 - beta)
    mse_beta = mean_squared_error(df["rating"], total_pred)
    mse.append((beta, mse_beta))

mse

[(0.1, 3.190707033152238),
 (0.2, 2.7537859399563884),
 (0.3, 2.392182812737872),
 (0.4, 2.105897651496697),
 (0.5, 1.894930456232864),
 (0.6, 1.7592812269463645),
 (0.7, 1.6989499636372052),
 (0.8, 1.7139366663053806),
 (0.9, 1.8042413349508999)]

In [15]:
betas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
mse = []
for beta in betas:
    total_pred = df["sentiment"] * beta + df["colab_rating"] * (1 - beta)
    mse_beta = mean_squared_error(df["rating"], total_pred)
    mse.append((beta, mse_beta))

mse

[(0.1, 2.1455533621270266),
 (0.2, 1.9483892546587809),
 (0.3, 1.7956359131533408),
 (0.4, 1.6872933376107033),
 (0.5, 1.6233615280308697),
 (0.6, 1.6038404844138403),
 (0.7, 1.628730206759613),
 (0.8, 1.6980306950681903),
 (0.9, 1.8117419493395714)]